In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/train/train.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
df.drop(columns='id', axis=1, inplace=True)

In [4]:
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [5]:
# Split Independent and Dependent Features

X = df.drop(columns='price')
y = df[['price']]

In [6]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [7]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [8]:
# Get numerical and categorical columns

numerical_columns = X.columns[X.dtypes != 'O']
categorical_columns = X.columns[X.dtypes == 'O']

In [9]:
# Define ranking for Ordinal Encoding

cut_categories = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

In [10]:
# Import libraries

from sklearn.impute import SimpleImputer # Handle Missing Values
from sklearn.preprocessing import StandardScaler # Handle Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Ranking

# Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [11]:
# Numerical Pipeline

num_pipeline = Pipeline(
    steps = [
    ('Imputer', SimpleImputer(strategy='median')),
    ('Scaler', StandardScaler())
    ]
)

In [12]:
# Categorical Pipeline

cat_pipeline = Pipeline(
    steps=[
    ('Imputer', SimpleImputer(strategy='most_frequent')),
    ('Ordinal_Encoder', OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories])),
    ('Scaler', StandardScaler())
    ]
)

In [13]:
# Pipeline

preprocessor = ColumnTransformer(
    [
    ('num_pipeline', num_pipeline, numerical_columns),
    ('cat_pipeline', cat_pipeline, categorical_columns)
    ]
)

In [14]:
# Split Data into Training and test Data
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=30)

In [15]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns = preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test), columns = preprocessor.get_feature_names_out())

In [16]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,0.874076,1.528722,1.352731
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,-2.144558,-0.935071,-0.646786
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,-0.132136,0.296826,0.686225
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,-0.132136,0.296826,0.019720
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,0.874076,2.144670,1.352731


In [17]:
X_test.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.564688,-0.942132,-0.642862,-0.429765,-0.464061,-0.500036,-0.132136,-0.935071,0.019720
1,-0.175556,1.000906,-0.121531,-0.042137,-0.028595,0.036132,-1.138347,0.912774,-0.646786
2,-1.061913,0.260701,-0.121531,-1.304180,-1.298703,-1.268060,0.874076,0.912774,2.685743
3,0.970223,-0.201927,1.963794,1.048629,0.996563,0.978049,-0.132136,0.296826,0.019720
4,-0.932202,-1.312235,0.399800,-1.006699,-0.990248,-1.065186,-0.132136,-0.935071,0.686225


In [18]:
# Model training

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet


In [19]:
import numpy as np

def evaluate_model(true, predicted):

    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(true, predicted)

    return mae, rmse, r2


In [21]:
models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'ElasticNet': ElasticNet()
}

model_list = list()
model_score = list()

for key, value in models.items():
    model = value
    model.fit(X_train, y_train)

    # make prediction
    y_pred = model.predict(X_test)

    mae, rmse, r2 = evaluate_model(y_test, y_pred)

    model_list.append(key)
    print(key)
    # print('\n')

    print('Model Training Performance: ')
    print('\tMean Absolute Error:', mae)
    print('\tRoot Mean Squared Error:', rmse)
    print('\tR2 Score:', (r2)*100)

    model_score.append(r2)
    print("="*35)
    # print('\n')

LinearRegression
Model Training Performance: 
	Mean Absolute Error: 674.0255115796832
	Root Mean Squared Error: 1013.9047094344004
	R2 Score: 93.68908248567512
Lasso
Model Training Performance: 
	Mean Absolute Error: 675.0716923362162
	Root Mean Squared Error: 1013.8784226767013
	R2 Score: 93.68940971841704
Ridge
Model Training Performance: 
	Mean Absolute Error: 674.0555800798253
	Root Mean Squared Error: 1013.9059272771639
	R2 Score: 93.68906732505938
ElasticNet
Model Training Performance: 
	Mean Absolute Error: 1060.7368759154729
	Root Mean Squared Error: 1533.4162456064048
	R2 Score: 85.56494831165182


In [22]:
for model, score in zip(model_list, model_score):
    print(f"Model: {model}, Score: {score}")

Model: LinearRegression, Score: 0.9368908248567511
Model: Lasso, Score: 0.9368940971841704
Model: Ridge, Score: 0.9368906732505938
Model: ElasticNet, Score: 0.8556494831165181
